# Project Title : Capstone V1
### Data Engineering Capstone Project

### Project Summary
##### The core purpose of the Capstone V1 project is just to help learners like us combine what we have learned throughout the program. This project will be an important part of our portfolio that will help us achieve the data engineering-related career goals.
##### In this project, we can choose to complete the project provided , or define the scope and data for a project with the own design using some provided data sources like Kaggle, Google Data Search, Data Gov,... 

### Project Goal
##### The core target of Capstone V1 project is to make a chance for those people or analytic groups such as data researchers, data analysts and other groups in order to analyze the large contexts of the US immigration data through gathering the data sources from four distinct datasets including:
    .Immigration.
    .Airport codes.
    .US Cities demographics. 
    .Global temperature. 
    .Through above dataset resources, the project built with a star schema which will enable data researchers, data analysts and other groups to get the insights of specific dataset. For instance, which city or area has more tourists visiting the United States, which states attract the most immigrant people, how long do they visit the US, what's the demography of the state where the immigrants see as the destination, and what's the average temparature of the US or other nations and the most common types of weather corresponding to each of temperature.

### Requirement: Working with four datasets to complete the project including:
        . "immigration_data_sample.csv" : This is the main dataset which including data on immigration to the United States.
        . "airport-codes_csv.csv" : The supplementary dataset will include data on airport codes of countries in some continents like NA, EU, AF, AS, OC.
        . "us-cities-demographics.csv" : This is also the supplementary dataset U.S. city demographics which provides the information about the demographics in some states, cities.
        . Lastly, the temperature data which describes the general information of temperature in some areas.  
    - In addition, also welcoming to enrich the project with some sources of additional data in case to set the own project apart.

### The "Capstone V1 " project consists of the following main steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Import the needed libraries
import pandas as pd
import numpy as np
import os
import configparser
import datetime as dt
import time

# Import the Pyspark resources
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear, avg, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.sql.functions import year, month, dayofmonth, weekofyear, date_format
from pyspark.sql import SparkSession, SQLContext, GroupedData, HiveContext
from pyspark.sql.functions import *
from pyspark.sql.functions import date_add as d_add
from pyspark.sql.types import DoubleType, StringType, IntegerType, FloatType
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql import Row

# Import other files for usage
import main as main 
import project_configuration as config 

### Step 1: Scope the Project and Gather Data

#### Scope 
- Project's goal : discover the insights of these datasets in the level of high-detail and make a project with a high level of applying to the real world in some cases. Besides, create a database that can be used as a resource for international immigrants and the US governement. 
- This project results could help policy makers make informed decisions on immigration that the goverments can control carefully the immigrantion people and confirm thier status like name, age, health and other personal information. In addition, it also help these people obtain a chance to stay in the US by using the valuable insights of the data for permanent accommodation here.
    
#### Describe and Gather Data 
##### The data usage consisting of : 4 main datasets (mentioned partly on the project requirement above)
##### Data Sources : 
    . "I94 Immigration Data": This data is from the US National Tourism and Trade Office. A data dictionary is included in the workspace. This is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in.
    . "World Temperature Data": This dataset came from Kaggle. 
    . "U.S. City Demographic Data": This data comes from OpenSoft. 
    . "Airport Code Table": This is a simple table of airport codes and corresponding cities.

In [2]:
#Define the needed resources of spark
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

immigration_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
immigration_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

## Reading Global Temperature Data

In [3]:
# Read Global Temperature Data using Pandas Library
imported_file = '../../data2/GlobalLandTemperaturesByCity.csv'
df_GlobalTemperature = pd.read_csv(imported_file)

In [4]:
#Showing the data type of the dataset
df_GlobalTemperature.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

In [5]:
# Displaying the DF Tuple
df_GlobalTemperature.shape

(8599212, 7)

In [6]:
# Picking the first tenth Global Temperature Rows
df_GlobalTemperature.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E
9,1744-08-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


### Reading AIRPORT DATA

In [7]:
# Read the Airport data using pandas library
provided_airport_data = 'airport-codes_csv.csv'
df_airport_data = pd.read_csv(provided_airport_data)

In [8]:
# Displaying the DF Tuple
df_airport_data.shape

(55075, 12)

In [11]:
#Cleaning the initial airport dataset
list_col = ['iata_code', 'local_code']
df_airport_data = main.remove_error_values(df_airport_data,list_col)

Starting the process of removing the rows containing the null values for the ['iata_code', 'local_code']
Before cleaning up, the total rows consists of 55075 rows.
After process of cleaning, the total rows include 2987 rows.


In [12]:
# Picking the first tenth Airport Data Rows
df_airport_data.head(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222"
440,07FA,small_airport,Ocean Reef Club Airport,8.0,NaN,US,US-FL,Key Largo,07FA,OCA,07FA,"-80.274803161621, 25.325399398804"
594,0AK,small_airport,Pilot Station Airport,305.0,NaN,US,US-AK,Pilot Station,NaN,PQS,0AK,"-162.899994, 61.934601"
673,0CO2,small_airport,Crested Butte Airpark,8980.0,NaN,US,US-CO,Crested Butte,0CO2,CSE,0CO2,"-106.928341, 38.851918"
1088,0TE7,small_airport,LBJ Ranch Airport,1515.0,NaN,US,US-TX,Johnson City,0TE7,JCY,0TE7,"-98.62249755859999, 30.251800537100003"
1402,13MA,small_airport,Metropolitan Airport,418.0,NaN,US,US-MA,Palmer,13MA,PMX,13MA,"-72.31140136719999, 42.223300933800004"
1438,13Z,seaplane_base,Loring Seaplane Base,0.0,NaN,US,US-AK,Loring,13Z,WLR,13Z,"-131.636993408, 55.6012992859"
1555,16A,small_airport,Nunapitchuk Airport,12.0,NaN,US,US-AK,Nunapitchuk,PPIT,NUP,16A,"-162.440454, 60.905591"
1574,16K,seaplane_base,Port Alice Seaplane Base,0.0,NaN,US,US-AK,Port Alice,16K,PTC,16K,"-133.597, 55.803"
1722,19AK,small_airport,Icy Bay Airport,50.0,NaN,US,US-AK,Icy Bay,19AK,ICY,19AK,"-141.662002563, 59.96900177"


### Reading US CITY DEMOGRAPHICS

In [15]:
# Read the US City Demographics data using pandas library
#imported_demographic_data = "us-cities-demographics.csv"
#df_demographics = pd.read_csv(imported_demographic_data, delimiter=';')
us_cities_dem_csv = "us-cities-demographics.csv"
df_demographics = pd.read_csv(us_cities_dem_csv, delimiter=';')
# Print the first five data rows
df_demographics.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [16]:
# Displaying the DF Tuple
df_demographics.shape

(2891, 12)

In [17]:
# Picking the first tenth US City Demographics Rows
df_demographics.head(10)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712.0,41971.0,80683,4815.0,8355.0,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629.0,56860.0,108489,3800.0,37038.0,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751.0,58077.0,109828,5204.0,16315.0,2.65,NC,Asian,11060


In [18]:
# Data Quality Checking after the process of data cleaning
for column in df_demographics:
    values = df_demographics[column].unique()
    if(True in pd.isnull(values)):
        print(f"The column {column} has null value")
print("Finish the process of checking for null data")   

The column Male Population has null value
The column Female Population has null value
The column Number of Veterans has null value
The column Foreign-born has null value
The column Average Household Size has null value
Finish the process of checking for null data


### Build country code to country name data frame

In [19]:
# Split and extract the two columns namely "Country Name" and "Country Code" from the provided data source "I94_SAS_Labels_Descriptions.SAS"
with open("I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()
    country_code = {}
    for countries in contents[10:298]:
        pair = countries.split('=')
        code,country = pair[0].strip(), pair[1].strip().strip("'")
        country_code[code] = country
df_country_code = pd.DataFrame(list(country_code.items()),columns=['code','country'])
# Picking the first tenth country data and the corresponding code.
df_country_code.head(10)

,code,country
0,236,AFGHANISTAN
1,101,ALBANIA
2,316,ALGERIA
3,102,ANDORRA
4,324,ANGOLA
5,529,ANGUILLA
6,518,ANTIGUA-BARBUDA
7,687,ARGENTINA
8,151,ARMENIA
9,532,ARUBA


### Pyspark Configuration

In [20]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [22]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
##### Identify data quality issues, like missing values, duplicate data, etc.
    . For the criteria like data quality issues that the method is just to make some creative ways to enhance the quality of the dataset such as add extra columns, rows if needed or even look for the more distinct suitable datasets on other data resources.
    . For some case like missing value, duplicate data, de-dupe data,... The stategy is just to remove and limit the column or row happing errors like that. 

#### Cleaning Steps
##### The necessary steps to clean the data including:
    . Step 1: Remove duplicate or irrelevant observations. Remove unwanted observations from your dataset, including duplicate observations or irrelevant observations. 
    . Step 2: Fix structural errors. 
    . Step 3: Filter unwanted outliers. 
    . Step 4: Handle missing data. 
    . Step 5: Validate and make the QA.

In [23]:
# Performing cleaning tasks 
immigration_spark = immigration_spark.drop("insnum","occup")

In [24]:
# clean up Immigration data
immigration_spark = immigration_spark.where(immigration_spark.arrdate.isNotNull())

In [25]:
immigration_spark = immigration_spark.where(immigration_spark.depdate.isNotNull())

In [ ]:
# clean up Global Temperature data
global_temperature_df = global_temperature_df.dropna()
# List out the some data rows for viewing
global_temperature_df.shape

In [ ]:
#discard historical data and work with data for past 20 years
print("Discarding historical data and work with data for recent 20 years")
dt_begin = "2000-01-01"
dt_end = "2019-01-01"
after_dt_begin = global_temperature_df["dt"] >= dt_begin
before_dt_end = global_temperature_df["dt"] < dt_end
dt_range = after_dt_begin & before_dt_end
global_temperature_df = global_temperature_df.loc[dt_range]
global_temperature_df.shape
global_temperature_df.head()

In [ ]:
# clean up Airport codes
column_list = ['iata_code', 'local_code']
airport_codes_df = main.cleanup_missing_column_values(airport_codes_df,column_list)
# List out the some data rows for viewing
airport_codes_df.shape

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
##### Utilizing the relational star schema for creating the database, and the core purpose of the DB foucuses on around the tables of immigrants with:
    . The Fact table : applying a star schema for this context. The fact table does not follow the obvious structure that all dimension tables will connect directly to the fact table. 
    . The Dimension table : showing the main context for immigration patterns that can give the US goverments more better decisions. 
    And these tables are shaped with each other and then truncated after finish a process. Lately, the core goal of helping the US government and International citizens address for the US immigration problems effectively.

#### 3.2 Mapping Out Data Pipelines
##### List the steps necessary to pipeline the data into the chosen data model: 
    . Create the high-level data pipeline flow for ETL process.
    . Design a detailed data pipeline utilizing the stength of AWS ecosystem.
    . Make a corresopnding for each steps in the data pipeline when implementing the ETL process.
    . Optimize the data pipeline flow as well as the ETL process for later usage.


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
##### Build the data pipelines to create the data model.
General architecture : Using the Apache Airflow for creating the data pipeline and utilizing the Apache Spark to pull the data and store them in a data lake on an S3 bucket in Parquet format.

In [ ]:
# Define immigration data
print(immigration_spark.count(),len(immigration_spark.columns))

# Design the schema
globaltemp_schema = StructType([StructField("dt", StringType(), True)\
                          ,StructField("average_temperature", FloatType(), True)\
                          ,StructField("average_temperature_uncertainty", FloatType(), True)\
                          ,StructField("city", StringType(), True)\
                          ,StructField("country", StringType(), True)\
                          ,StructField("latitude", StringType(), True)\
                          ,StructField("longitude", StringType(), True)])
global_temperature_df.rename(columns={'AverageTemperature':'average_temperature'}, inplace=True)
global_temperature_df.rename(columns={'AverageTemperatureUncertainty':'average_temperature_uncertainty'}, inplace=True)
global_temperature_df.rename(columns={'City':'city'}, inplace=True)
global_temperature_df.rename(columns={'Country':'country'}, inplace=True)
global_temperature_df.rename(columns={'Latitude':'latitude'}, inplace=True)
global_temperature_df.rename(columns={'Longitude':'longitude'}, inplace=True)

temp_spark = spark.createDataFrame(global_temperature_df, schema=globaltemp_schema)

temp_spark.toPandas().head()


In [ ]:
dem_schema = StructType([StructField("City", StringType(), True)\
                        ,StructField("State", StringType(), True)\
                        ,StructField("Median Age", FloatType(), True)\
                        ,StructField("Male Population", FloatType(), True)\
                        ,StructField("Female Population", FloatType(), True)\
                        ,StructField("Total Population", IntegerType(), True)\
                        ,StructField("Number of Veterans", FloatType(), True)\
                        ,StructField("Foreign-born", FloatType(), True)\
                        ,StructField("Average Household Size", FloatType(), True)\
                        ,StructField("State Code", StringType(), True)\
                        ,StructField("Race", StringType(), True)\
                        ,StructField("Count", IntegerType(), True)])

dem_spark = spark.createDataFrame(demographics_df, schema=dem_schema)
dem_spark.toPandas().head()

In [ ]:
# Show the airport schema
airport_schema =  StructType([StructField("ident", StringType(), True)\
                        ,StructField("type", StringType(), True)\
                        ,StructField("name", StringType(), True)\
                        ,StructField("elevation_ft", FloatType(), True)\
                        ,StructField("continent", StringType(), True)\
                        ,StructField("iso_country", StringType(), True)\
                        ,StructField("iso_region", StringType(), True)\
                        ,StructField("municipality", StringType(), True)\
                        ,StructField("gps_code", StringType(), True)\
                        ,StructField("iata_code", StringType(), True)\
                        ,StructField("local_code", StringType(), True)\
                        ,StructField("coordinates", StringType(), True)])
airport_spark = spark.createDataFrame(airport_codes_df, schema=airport_schema)
# Using pandas for viewing some data rows.
airport_spark.toPandas().head()

### Create a path storing the output 

In [ ]:
# Define the directory of output
output_directory="destination_table/"

In [ ]:
# Define airport dimension
dim_airport = helper.create_airport(airport_spark, output_path)

# Define status dimension
dim_status = helper.create_status(immigration_spark,output_path)

# Define status dimension
dim_time = helper.create_time(immigration_spark,output_path)

# Define visa dimension
dim_visa = helper.create_visa(immigration_spark,output_path)

# Define country dimension
dim_country= spark.createDataFrame(country_code_df)
country = helper.create_country(dim_country,output_path)

# Define temperature dimension
temperature = create_temperature(temp_spark,output_path)


#### Create Immigration Fact

In [ ]:
# join city dimension and temperature dimension
country_temperature = country.select(["*"])\
            .join(temperature, (country.country == upper(temperature.country)), how='full')\
            .select([country.code, country.country, temperature.temperature_id, temperature.average_temperature, temperature.average_temperature_uncertainty])

country_temperature.write.mode("overwrite").parquet(output_path+"country_temperature_mapping")

# Define the immigration data
immigration = helper.create_immigration(immigration_spark, output_path, spark)

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing : create some realted unit tests for specific files in order to evaluate the data quality.  
 * Source/Count checks to ensure completeness : apply the Quality checks some situations like (duplicate, unique and no-null data) 
 
Run Quality Checks

In [ ]:
# Define a genneric data examining for airport data
airport = spark.read.parquet(output_path+"airport")
airport.toPandas().dtypes

# Define a genneric data examining for status data
status = spark.read.parquet(output_path+"status")
status.toPandas().dtypes

# Define a genneric data examining for time data
time = spark.read.parquet(output_path+"time")
time.toPandas().dtypes

# Define a genneric data examining for visa data
visa = spark.read.parquet(output_path+"visa")
visa.toPandas().dtypes

# Define a genneric data examining for temperature data
temperature = spark.read.parquet(output_path+"temperature")
temperature.toPandas().dtypes

# Define a genneric data examining for country data
country = spark.read.parquet(output_path+"country")
country.toPandas().dtypes

# Define a genneric data examining for immigration data
immigration = spark.read.parquet(output_path+"immigration")
immigration.toPandas().dtypes

In [ ]:
# Perform quality checks whether the records are completed for every destination tables defined above. 
main.run_record_count_check(airport, "airport")
mani.run_record_count_check(time, "time")
main.run_record_count_check(status, "status")
main.run_record_count_check(visa, "visa")
main.run_record_count_check(state, "state")
main.run_record_count_check(temperature, "temperature")
main.run_record_count_check(immigration, "immigration")

#### 4.3 Data dictionary 
Create a data dictionary for the data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

##### Using the "Data Dictionary file" for witnessing more about the case of data dictionary.

#### Step 5: Complete Project Write Up
##### Clearly state the rationale for the choice of tools and technologies for the project:
    . Take the advantages of AWS ecosystem with lots of useful and effective services for optimizing a vast of the implementing time and budget.
    . Utilizing the technologies namley Apache Spark (Pyspark) AWS S3, EMR, EC2, and AWS Redshift.
    . AWS EMR (with EC2) : just due to the fact that the spark dependencies come preloaded and it helps to prevent the action of manually set up the dependencies on each cluster and be easier to add more packages and fucntionality if needed.
    . AWS S3 : it can bring the lucrative case for the goverment and restore more high-capacity resources. And using the S3 bucket can help me to save memory space and allocation via storing the processed files in parquet format.
    . AWS Redshift : helps the related people to access the data resources when needed. Besides, using Redshift take a greater price of usage, and it can help in making the relevant settings and other configurations when implementing.
    
* ##### Propose how often the data should be updated and why:
    . The data should be updated every quarters because the data quantity will be increased sharply via hours or even seconds with a great deal of one, so that it really needs to update the data with this assigned schedule.
    
* ##### Write a description of how you would approach the problem differently under the following scenarios:
 * ##### The data was increased by 100x.
     . Applying the AWS Redshift can handle big reads and writes so there wouldn't need to make some changes when allocating more nodes for the emr cluster to scale with the data quantity.
     
 * ##### The data populates a dashboard that must be updated on a daily basis by 7am every day.
     . For solving this requirement, definitely needing to utilize AWS EventBrigde or maybe the Airflow and have a daily notification when implementing on 7am every day and then send the results via email about the status of the data process.
     
 * ##### The database needed to be accessed by 100+ people.
     . Utilizing the AWS redshift will handle effectively this senario with satisfying +100 people at the same time with the must-have condition that these people have to own the correct AWS credentials like IAM key and secret key, so that these people would be able to implement their own exploration and reports when needed.